## 常见类和对象操作

这些从实践中观察如何来构造常见的类的示例，包括但不限于
1. torch dataset 的自定义实现方式，【基于 tensor】、【基于序列数据】、【基于文本数据】
2. torch Model 的自定义实现方式
3. scikit learner 的自定义实现方式
4. 定义数据预处理的管道（Filter、Transformer、Fit、Predict）
5. 自定义评估指标和回调函数
6. 创建工具类类封装常用的功能，包括日志记录、配置管理等
7. 利用类关管理实验的参数、结果和模型、方便实验的追踪和对比
8. 集成第三方服务和接口，利用 API 封装来简化调用

In [9]:
# Py Torch 中的 dataset 和 dataloader 
# 如何通过自己编写脚本的方式来实现数据集的自身制作
import torch 
from torch.utils.data import Dataset, DataLoader

class BaseDataset(Dataset):
    ''' 
    通过集成 torch.utils.data.Dataset 类来实现自定义数据集
    '''
    def __init__(self):
        self.data = torch.randn(100, 3, 32, 32)
        self.label = torch.randint(0, 10, (100, ))
    
    def __len__(self):
        ''' 
        重新定义两个基本的方法，一个是 __len__ 方法，用来返回数据集的长度
        '''
        return len(self.data)
    
    def __getitem__(self, index):
        '''
        另一个是 __getitem__ 方法，用来根据索引获取数据
        '''
        return self.data[index], self.label[index]

class SeriesDataset(Dataset):
    ''' 
    自定义适合时间序列数据的数据集
    '''
    def __init__(self, data, window_size):
        self.data = data
        self.window_size = window_size
    
    def __len__(self):
        return len(self.data) - self.window_size
    
    def __getitem__(self, index):
        return self.data[index:index+self.window_size]

class TextDataset(Dataset):
    ''' 
    自定义适合文本数据的数据集
    '''
    def __init__(self, data, vocab):
        self.data = data
        self.vocab = vocab
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]

In [10]:
#  callback 函数是什么意思？ 
# 回调是编程模式，讲一个函数作为参数传递给另外一个函数，再满足某个特定条件发生时调用该
# 函数，这种机制允许程序再运行过程中动态决定某些操作，从而实现灵活性和拓展性

# 回调函数的分类包括
# 1. 同步回调 sync callback
# 2. 异步回调 async callback,这里的操作和嵌入式中的中断很相似



['pytorch', 'hello']


In [13]:
import requests
import threading 

def fetch_url(url, callback):
    '''
    通过 requests 库来获取 url 的内容
    '''
    def run():
        response = requests.get(url)
        callback(response)
    threading.Thread(target=run).start()

def print_response(response):
    '''
    打印 response 的内容
    '''
    print(response.text)

print('start fetching')
fetch_url('https://www.baidu.com', print_response)
print('end fetching')

start fetching
end fetching


<!DOCTYPE html>
<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;charset=utf-8><meta http-equiv=X-UA-Compatible content=IE=Edge><meta content=always name=referrer><link rel=stylesheet type=text/css href=https://ss1.bdstatic.com/5eN1bjq8AAUYm2zgoY3K/r/www/cache/bdorz/baidu.min.css><title>ç¾åº¦ä¸ä¸ï¼ä½ å°±ç¥é</title></head> <body link=#0000cc> <div id=wrapper> <div id=head> <div class=head_wrapper> <div class=s_form> <div class=s_form_wrapper> <div id=lg> <img hidefocus=true src=//www.baidu.com/img/bd_logo1.png width=270 height=129> </div> <form id=form name=f action=//www.baidu.com/s class=fm> <input type=hidden name=bdorz_come value=1> <input type=hidden name=ie value=utf-8> <input type=hidden name=f value=8> <input type=hidden name=rsv_bp value=1> <input type=hidden name=rsv_idx value=1> <input type=hidden name=tn value=baidu><span class="bg s_ipt_wr"><input id=kw name=wd class=s_ipt value maxlength=255 autocomplete=off autofocus=autofocus></span><sp

In [8]:
## 第三方 API 的使用
import requests

class GaodeAPIClient:
    def __init__(self, key):
        self.key = key
    
    def get_location(self, address):
        ''' 
        获取地址的经纬度
        '''
        url = 'https://restapi.amap.com/v3/geocode/geo'
        params = {
            'key': self.key,
            'address': address
        }
        response = requests.get(url, params=params)
        data = response.json()
        return data['geocodes'][0]['location']
    
    def get_distance(self, origin, destination):
        ''' 
        获取两个地址之间的距离
        '''
        url = 'https://restapi.amap.com/v3/distance'
        params = {
            'key': self.key,
            'origins': origin,
            'destination': destination
        }
        response = requests.get(url, params=params)
        data = response.json()
        return data['results'][0]['distance']
    
    def get_path_transport(self,origin,destination,type = 'bus'):
        ''' 
        获取不同交通方式下的交通路径，可选的包括驾车、公交、步行、骑行
        这里的 API 需要考虑再优化一下
        '''
        url = 'https://restapi.amap.com/v3/direction/'+type
        params = {
            'key': self.key,
            'origin': origin,
            'destination': destination
        }
        response = requests.get(url, params=params)
        data = response.json()
        return data

with open('key.xcl', 'r') as f:
    key = f.read().strip()
data_client = GaodeAPIClient(key=key)
location = data_client.get_location('北京市朝阳区望京SOHO')
print(location)
distance = data_client.get_distance('116.482038,39.997621', '116.313393,39.984092')
print(distance)
path = data_client.get_path_transport('116.482038,39.997621', '116.313393,39.984092')
print(path)

116.480639,39.996356
17621
{'status': '0', 'info': 'SERVICE_NOT_AVAILABLE', 'infocode': '10002'}
